In [2]:
import numpy as np
import pandas as pd
import csv

In [47]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
artists = pd.read_csv('artists.csv')
profiles = pd.read_csv('profiles.csv')

In [59]:
merged = pd.merge(train_init, profiles, on='user')

In [60]:
merged.describe(include='all')

,user,artist,plays,sex,age,country
count,4154804,4154804,4154804.000000,3809426,3362501.000000,4154804
unique,233286,2000,NaN,2,NaN,239
top,4ddafccadfb57dfd1096f615ed67816c41512464,a74b1b7f-71a5-4011-9441-d0b5e4122711,NaN,m,NaN,United States
freq,47,34069,NaN,2743959,NaN,897641
mean,NaN,NaN,253.016796,NaN,24.448320,NaN
std,NaN,NaN,717.096328,NaN,21.548704,NaN
min,NaN,NaN,1.000000,NaN,-1337.000000,NaN
25%,NaN,NaN,48.000000,NaN,20.000000,NaN
50%,NaN,NaN,118.000000,NaN,23.000000,NaN
75%,NaN,NaN,264.000000,NaN,27.000000,NaN


In [65]:
len(train)

4154804

In [66]:
len(test)

4154804

In [63]:
merged = pd.merge(train,test,on='user')

In [64]:
len(set(merged['user']))

233286